In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [2]:
!pip install streamlit
!pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [7]:
%%writefile app.py
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
import streamlit as st
import os
import PyPDF2  # Pour l'extraction de texte depuis PDF

# LOGIN HUGGING FACE
login(token="")

# CONFIGURATION DU MODELE LLaMA
model_name = "meta-llama/Llama-3.2-3B"  # Remplacer par le modèle LLaMA approprié
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,  # Longueur maximale des tokens en entrée
    num_beams=5,
    pad_token_id=tokenizer.pad_token_id
)
llm = HuggingFacePipeline(pipeline=pipe)

# CONFIGURATION DES EMBEDDINGS
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# EXTRACTION DE TEXTE DU PDF
def extract_text_from_pdf(pdf_path):
    """Extrait le texte d'un fichier PDF."""
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# TRONQUER LE CONTEXTE AVEC LE TOKENIZER
def truncate_with_tokenizer(text, tokenizer, max_length=512):
    """Tronque le texte pour respecter la longueur maximale en utilisant le tokenizer."""
    tokenized = tokenizer(text, truncation=True, max_length=max_length, return_tensors="pt")
    return tokenizer.decode(tokenized["input_ids"][0], skip_special_tokens=True)

# DIVISER LE CONTEXTE EN MORCEAUX
def split_context_into_chunks(context, max_chunk_size=512):
    """Divise le contexte en morceaux pour respecter la taille maximale."""
    tokens = context.split()
    chunks = [" ".join(tokens[i:i + max_chunk_size]) for i in range(0, len(tokens), max_chunk_size)]
    return chunks

# CRÉATION OU CHARGEMENT DE FAISS
pdf_path = "data/FA208976_20241120_163611.pdf"  # Remplacer par le chemin réel de votre PDF
faiss_index_path = "faiss_index"

if not os.path.exists(faiss_index_path):
    st.info("Extraction du texte et création de l'index FAISS...")
    pdf_text = extract_text_from_pdf(pdf_path)  # Extraire le texte du PDF

    # Découper le texte en morceaux
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_text(pdf_text)

    # Créer l'index FAISS
    vectorstore = FAISS.from_texts(texts, embeddings)
    vectorstore.save_local(faiss_index_path)
else:
    st.info("Chargement de l'index FAISS existant...")
    vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever()

# TEMPLATE POUR LES QUESTIONS-RÉPONSES
qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""Utilisez le contexte suivant pour répondre de manière précise et concise à la question posée.
    Si le contexte ne contient pas d'information pertinente, indiquez "Information non disponible dans le contexte".
    Contexte : {context}
    Question : {question}
    Réponse : """
)

# CHAINE LLM POUR QUESTIONS-RÉPONSES
qa_chain = LLMChain(llm=llm, prompt=qa_prompt)

# INTERFACE STREAMLIT
st.title("Système de Questions-Réponses avec PDF et LLaMA")
st.header("Posez des questions basées sur le contenu du PDF")

# Entrée utilisateur
question = st.text_input("Entrez votre question ici :")

if st.button("Soumettre"):
    if question.strip():
        # Récupérer les documents pertinents
        retrieved_docs = retriever.get_relevant_documents(question)
        context = " ".join([doc.page_content for doc in retrieved_docs])

        if not context.strip():
            st.warning("Aucune information pertinente trouvée dans le fichier PDF.")
        else:
            chunks = split_context_into_chunks(context, max_chunk_size=512)
            responses = []

            for chunk in chunks:
                # Tronquer le chunk avec le tokenizer
                truncated_chunk = truncate_with_tokenizer(chunk, tokenizer, max_length=512)

                # Vérifier la longueur des tokens
                token_count = len(tokenizer(truncated_chunk)["input_ids"])
                if token_count > 512:
                    st.error(f"Un chunk dépasse toujours la limite des tokens : {token_count} tokens.")
                    continue

                # Passer au modèle
                response = qa_chain.run(context=truncated_chunk, question=question)
                responses.append(response)

            # Combiner les réponses
            final_response = " ".join(responses)

            st.success("Réponse :")
            st.write(final_response)
    else:
        st.warning("Veuillez entrer une question avant de soumettre.")

st.sidebar.info("Cette application utilise un fichier PDF, FAISS et un modèle LLaMA pour répondre aux questions.")


Overwriting app.py


In [4]:
!wget -q -O - ipv4.icanhazip.com

35.233.251.0


In [8]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹

⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.233.251.0:8501

your url is: https://red-kids-mate.loca.lt
/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py:549: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import HuggingFacePipeline`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/content/app.py:4: LangChainDeprecationWarning: Importing FAISS from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import FAISS

with new imports of:

>> from langchain_community.vectorstores import FAISS
You can use the langchain cli to **automatically** upgrade many imports. Please see docum